<a href="https://colab.research.google.com/github/GerardoMunoz/embedded/blob/main/MultiTask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Multitasking Strategies in Embedded Systems and RTOS

## Multitasking Strategies in Embedded Systems

Multitasking in embedded systems allows multiple tasks to run concurrently or appear to run at the same time.

### 1. **Superloop (Main Loop)**
The simplest model: all tasks run sequentially in an infinite loop.

**Advantages**
- Easy to implement
- Very low memory usage

**Disadvantages**
- No priority handling
- Not suitable for strict real-time applications

---

### 2. **Cooperative Multitasking**
Each task runs and **voluntarily yields** CPU control.

**Advantages**
- Simple design
- Low overhead
- No unexpected task interruptions

**Disadvantages**
- If a task does not yield → system can freeze
- Not ideal for critical real-time tasks

---

### 3. **Interrupt-Driven Multitasking**
Interrupts trigger specific tasks when external or timer events occur.

**Advantages**
- Fast response to critical events
- Efficient event handling

**Disadvantages**
- Becomes complex with many interrupts
- ISRs must remain short to avoid latency issues

---

##  Multitasking Strategies in RTOS

An **RTOS (Real-Time Operating System)** uses a scheduler to guarantee predictable timing and manage task switching.

### 1. **Preemptive Multitasking**
The RTOS interrupts a task to run another one with higher priority.

**Advantages**
- Guaranteed time-critical response
- Ideal for systems with many concurrent tasks

**Disadvantages**
- More complex
- Higher resource usage

---

### 2. **Cooperative Multitasking in RTOS**
Tasks yield control voluntarily, but managed by the RTOS.

**Advantages**
- Lower scheduling overhead
- More predictable control points

**Disadvantages**
- If a task misbehaves, timing may suffer
- Not recommended for safety-critical systems

---

### 3. **Scheduling Algorithms in RTOS**

| Algorithm | Description |
|----------|-------------|
| **Round-Robin** | Shares CPU time equally among tasks of the same priority |
| **Priority-Based** | Highest-priority task runs first |
| **Rate-Monotonic (RMS)** | Priority assigned based on task frequency |
| **Earliest-Deadline-First (EDF)** | Task with the nearest deadline runs first |

---

##  Summary Table

| Strategy | Type | Predictability | Complexity | Typical Use |
|---------|------|----------------|-----------|-------------|
Superloop | Sequential | Low | Very low | Simple embedded devices |
Cooperative | Non-RTOS / RTOS | Medium | Low | Non-critical systems |
Interrupt-Driven | Event-based | High | Medium-High | Reactive and sensor systems |
RTOS Preemptive | Real-time | Very high | High | Automotive, industrial, medical |

---

## ✅ RTOS vs Non-RTOS in Embedded Design


| Feature                  | RTOS (Real-Time OS)                                                             | Non-RTOS / Bare-Metal                                        |
| ------------------------ | ------------------------------------------------------------------------------- | ------------------------------------------------------------ |
| **Task Scheduling**      | Deterministic scheduling (preemptive or cooperative), tasks managed by priority | Loop-based or interrupt-based; manual task control           |
| **Timing Guarantees**    | Hard or soft real-time guarantees                                               | Not guaranteed; depends on developer design                  |
| **Multitasking**         | True multitasking with context switching                                        | Sequential execution or cooperative execution                |
| **Latency**              | Predictable low latency                                                         | Can vary; dependent on code structure and ISR handling       |
| **Memory & Resources**   | Requires more RAM/Flash                                                         | Minimal memory usage                                         |
| **Complexity**           | Higher software complexity                                                      | Simple and direct control                                    |
| **Debugging**            | Advanced debugging tools (task aware)                                           | Basic debugging; harder for complex systems                  |
| **Scalability**          | Easy to scale to many tasks                                                     | Hard to scale beyond a few tasks                             |
| **Power Consumption**    | Can efficiently sleep tasks, idle hooks                                         | Depends on architecture; developer must manage power         |
| **Typical Applications** | Robotics, automotive, drones, industrial automation, medical devices            | Simple sensor controllers, IoT nodes, small appliances, toys |

## `micropython.schedule()` — Summary

`micropython.schedule(callback, arg)` allows you to safely defer work from an interrupt handler (ISR) to the main MicroPython thread.
It queues a function to run as soon as interrupts are re-enabled and Python execution resumes, avoiding long or unsafe ISR code.

### Purpose
* Execute code outside the interrupt context
* Keep ISRs short and fast
* Avoid crashes or timing issues caused by heavy work inside interrupts

### When to use
Use micropython.schedule() when an interrupt needs to trigger non-trivial work, such as:

- Updating application logic
- Printing or logging
- Changing shared data structures
- Interacting with MicroPython objects

### ISR Rules

Inside an ISR do NOT:

- Call print()
- Allocate large memory
- Use blocking operations (sleep, I/O)
- Modify complex Python objects

Instead:

- Read minimal data
- Set a flag or call micropython.schedule()

### Signature:

`micropython.schedule(callback, arg)`

- callback: a Python function to run later
- arg: a small integer/object passed to the callback

### Example
```python
import micropython
from machine import Pin

def deferred_task(arg):
    print("Button pressed! arg =", arg)

def irq_handler(pin):
    micropython.schedule(deferred_task, 1)

button = Pin(15, Pin.IN, Pin.PULL_UP)
button.irq(trigger=Pin.IRQ_FALLING, handler=irq_handler)
```

### Official Docs

- https://docs.micropython.org/en/latest/reference/isr_rules.html#micropython-schedule-function

## Memory in threads

You can safely allocate memory in both threads — but only if you coordinate access and avoid doing it inside ISR context.

MicroPython on the Pico 2 W uses global interpreter lock–style cooperative threading, so both threads share:

- The same heap
- The same Python VM
- The same garbage collector

That means:

Memory allocation from two threads is safe only if you avoid simultaneous GC and protect shared data.

### Safe way to allocate memory from 2 threads


Rules
| Safe practice                                       | Why                      |
| --------------------------------------------------- | ------------------------ |
| `_thread.allocate_lock()` around shared structures  | prevents data corruption |
| Avoid memory allocation in ISRs                     | GC isn't ISR-safe        |
| Avoid large allocations in tight loops              | may trigger GC           |
| Call `gc.collect()` at controlled points            | predictable memory       |
| Prefer pre-allocated buffers for time-critical code | avoids fragmentation     |

### Example of using lock

```python
import _thread
import gc

lock = _thread.allocate_lock()
shared_list = []

def thread1():
    global shared_list
    while True:
        with lock:               # Safe memory allocation
            shared_list.append(1)
        gc.collect()             # optional

def thread2():
    global shared_list
    while True:
        with lock:
            shared_list.append(2)
        gc.collect()            # optional

_thread.start_new_thread(thread1, ())
thread2()
```

### What can go wrong
| Risk                 | Cause                                           |
| -------------------- | ----------------------------------------------- |
| Heap corruption      | modifying shared objects without lock           |
| Hard freezes         | GC triggered while memory use in another thread |
| Race conditions      | scheduling not time-deterministic               |
| Missed sensor events | Python thread overhead + timing jitter          |


So if you allocate memory in both threads without locks, eventual crash is likely.

### Best pattern (for robotics / real-time)

- Pre-allocate large camera buffers
- Use ring buffers, not append()




## Python `list` in multitask

Pushing data into a Python list can allocate memory.

MicroPython lists grow dynamically. When you do:

```my_list.append(x)```


MicroPython may need to allocate memory to expand the list storage if it's full.
That means:

- It might not allocate (if capacity exists)

- It can allocate (if resizing needed)

And allocation inside an ISR is unsafe — it can crash, corrupt memory, or stall execution.

So in an ISR, you should not rely on `list.append()`.

### Safe alternative in ISR: pre-allocated ring buffer

Create a fixed-size buffer in advance, then write into it by index:

```python
BUFFER_SIZE = 32
ring = [None] * BUFFER_SIZE
head = 0

def irq_handler(pin):
    global head, ring
    ring[head] = pin.value()  # or some small integer/event code
    head = (head + 1) % BUFFER_SIZE
```

This uses no dynamic allocation, so it’s ISR-safe.

### What data types are safe to store?

Inside ISR, store only simple, pre-existing objects, e.g.:

- ✅ small ints
- ✅ references to existing objects
- ❌ strings you construct
- ❌ lists, dicts, tuples you build at runtime

Creating new objects = memory allocation = unsafe

### Rule of thumb

| Operation                | ISR-safe?          | Why                   |
| ------------------------ | ------------------ | --------------------- |
| `list.append()`          | ❌ not guaranteed   | list may grow → alloc |
| `buffer[index] = value`  | ✅                  | no allocation         |
| `micropython.schedule()` | ✅                  | defers work safely    |
| `queue.put()`            | ⚠️ usually NO      | may allocate          |
| using `deque`            | ❌                  | dynamic               |
| C array / array module   | ✅ if pre-allocated | static memory         |


### Example: ISR + scheduled handler + queue (safe pipeline)

```python
import micropython

BUFFER_SIZE = 32
ring = [0] * BUFFER_SIZE
head = 0

def irq(pin):
    global head
    ring[head] = 1  # event flag
    head = (head + 1) % BUFFER_SIZE
    micropython.schedule(process_events, 0)

def process_events(_):
    # Read ring buffer here safely outside ISR
    ...
```

### Summary
| Action                                        | Safe in ISR?        |
| --------------------------------------------- | ------------------- |
| Appending to list                             | ❌ No — may allocate |
| Writing into pre-allocated array              | ✅ Yes               |
| Scheduling work with `micropython.schedule()` | ✅ Yes               |



## What is _thread.allocate_lock()?

It returns a mutex lock object used for thread synchronization.

### Purpose

* Prevent two threads from modifying shared data at the same time
* Avoid race conditions and memory corruption

### Type

It's a lock object (similar to a mutex in C).
In MicroPython / CPython, the type is _thread.lock

### What can it do?

| Method                | Meaning                                                               |
| --------------------- | --------------------------------------------------------------------- |
| `lock.acquire()`      | Take the lock (wait if already taken)                                 |
| `lock.release()`      | Release the lock                                                      |
| `lock.acquire(False)` | Try to take the lock **without blocking** (returns `True` or `False`) |





### Protect boolean shared between threads
```python
import _thread

lock = _thread.allocate_lock()
shared_flag = False

def thread_func():
    global shared_flag
    while True:
        with lock:
            shared_flag = not shared_flag  # safe modification
        # do work...

_thread.start_new_thread(thread_func, ())

# main thread
while True:
    with lock:
        print("Flag =", shared_flag)
```

Here:

with lock: locks access

Inside that block, boolean changes are safe

### Checking if lock is free (non-blocking attempt)
```python
if lock.acquire(False):
    # got the lock
    try:
        shared_flag = True
    finally:
        lock.release()
else:
    # lock busy, skip
    pass
```

This is useful when you do not want to block the thread.


### When does the lock change state?

| Moment                   | Lock state             |
| ------------------------ | ---------------------- |
| Thread calls `acquire()` | Lock becomes **taken** |
| Thread calls `release()` | Lock becomes **free**  |


## Complex and simpler I/O

You should NOT perform complex I/O in a MicroPython ISR, even if it is “non-blocking.”

### Why?

Even “non-blocking” I/O in Python is not guaranteed to be interrupt-safe.
In MicroPython, I/O typically still involves:

- Python object manipulation
- Buffer management
- Locks or internal state
- Potential memory allocation
- Driver code that may block briefly

All of these are unsafe inside an interrupt handler (ISR).

MicroPython ISRs must be very fast, deterministic, and allocation-free.
Non-blocking socket ops may seem safe, but they can still:

- Trigger memory allocation
- Interact with internal MicroPython scheduler
- Stall due to underlying RTOS/netstack rules
- Cause race conditions or crashes

So they fall under "no I/O in ISR".

### What can you do in an ISR

Inside an ISR you may:

| Allowed                       | Notes                     |
| ----------------------------- | ------------------------- |
| Set a flag                    | simplest and safest       |
| Write to hardware registers   | timer, GPIO toggles, etc. |
| Read simple input values      | pin, ADC snapshot         |
| Use `micropython.schedule()`  | best practice             |
| Push event into a ring buffer | pre-allocated only        |

### What NOT to do in ISR
| Avoid                     | Why                  |
| ------------------------- | -------------------- |
| `sleep()`                 | blocks system        |
| `print()`                 | allocates & delays   |
| socket operations         | unsafe & may block   |
| file I/O                  | may allocate / block |
| dynamic memory operations | unpredictable        |
| long loops / math         | delays whole system  |

### Correct Pattern For Networking ISR

Example: You want to trigger a socket read when a GPIO event happens.

```python
def irq_handler(pin):
    micropython.schedule(handle_event, None)

def handle_event(_):
    # Now safe to call socket functions
    try:
        data = sock.recv(1024)  # non-blocking ok here
        print("Got data:", data)
    except:
        pass
```


In other words:

ISR → schedule callback → callback does non-blocking socket work


### Key Principle

ISR: signal only
Main loop / scheduled callback: do real work

Even if the socket call is non-blocking, put it outside the ISR.

### Summary
Question    Answer
Can I do non-blocking socket calls in ISR?  ❌ No
Why not?    Python/network stack may allocate or stall
Correct method? ✅ micropython.schedule() → network code


## Conclusion

- Pre-allocate all fixed-size variables and buffers before starting threads or enabling interrupts to avoid runtime memory allocation issues and timing instability.
- Use `micropython.schedule(callback, arg)` to defer non-trivial work triggered by interrupts, ensuring interrupt service routines remain short and deterministic.

```python
import machine, _thread, micropython, time

# -------------------------------------------------
# Configuration
# -------------------------------------------------
PIN_BUTTON = 1
PIN_LED = "LED"

BUFFER_SIZE = 64  # ring buffer for timestamps
buffer = [0] * BUFFER_SIZE
head = 0
tail = 0

button = machine.Pin(PIN_BUTTON, machine.Pin.IN, machine.Pin.PULL_UP)
led = machine.Pin(PIN_LED, machine.Pin.OUT)


# -------------------------------------------------
# ISR: GPIO interrupt → store timestamp in buffer
# -------------------------------------------------
def button_irq(pin):
    global head, buffer
    t = time.ticks_us()  # timestamp in microseconds

    # store in ring buffer (NO allocation)
    buffer[head] = t
    head = (head + 1) % BUFFER_SIZE

# -------------------------------------------------
# Deferred printing function (scheduled)
# -------------------------------------------------
def print_buffer(_):
    global buffer, head, tail

    tmp = head  # safe snapshot: head is an immutable int; if it were mutable we'd need a lock or copy()


    if tail <= tmp:
        data = buffer[tail:tmp]
    else:
        data = buffer[tail:] + buffer[:tmp]

    print("New timestamps ({} events):".format(len(data)))
    print(data)

    tail = tmp
    print("------------------")

# -------------------------------------------------
# ISR: Timer interrupt → schedule print
# -------------------------------------------------
def timer_cb(t):
    micropython.schedule(print_buffer, 0)

# -------------------------------------------------
# Thread on "Core 1"
# Toggle LED based on button state (debounced read)
# -------------------------------------------------
def core1_thread():

    state = 1
    while True:
        new_state = button.value()
        if new_state != state:
            state = new_state
            led.value(not state)   # LED ON when button pressed (active low)
        time.sleep_ms(100)

_thread.start_new_thread(core1_thread, ())

# -------------------------------------------------
# Hardware setup
# -------------------------------------------------
button.irq(trigger=machine.Pin.IRQ_FALLING | machine.Pin.IRQ_RISING,
           handler=button_irq)

# Timer: print the buffer every second
timer = machine.Timer()
timer.init(period=1000, callback=timer_cb)

print("System running. Press the button to see interrupt noise.")


```

## Project Task

### Core 1 / Worker Thread

- Start a dedicated thread (intended to run on Core 1)
- Capture video from the OV7670 camera at 40×30 RGB565
- Process each frame to compute Lucas–Kanade optical flow for four tracking points
- Publish a PubSub topic indicating:
- The frame has been captured
- The velocity vectors calculated from optical flow

### Core 0 / Main Control

- Use hardware timer interrupts to read sensor values (encoders, IMU, distance sensors)
- In interrupts, only set flags and trigger micropython.schedule() calls
- Scheduled tasks update servo and motor commands at a fixed control frequency

- Handle network communication (client and server **non-blocking** sockets)
- Use PubSub to exchange data between threads and system components

### Concurrency Rules

- Interrupts do not perform logic or comolex I/O, instead they trigger scheduled callbacks
- Shared data structures protected with locks or message queues
- Camera processing and control loop run concurrently